In [ ]:
from dotenv import load_dotenv
import os
import time
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np 
from Preprocessing.preprocessing_pipeline_final import preprocessing_pipeline

load_dotenv()  # lädt .env automatisch
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")



In [ ]:
def call_llm(text):
    # Diese Funktion nutzt die API von Google um sich gegen das LLM zu schalten. Prompt etc. ist unten zu sehen

    llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
    )

    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                '''Du bist ein Experte für Automobilklassifikation. Deine Aufgabe ist es, Automodelle basierend auf ihrem Namen einem von fünf vordefinierten Fahrzeugsegmenten zuzuordnen. Bitte gib nur den Namen des passenden Segments zurück.

                    Die Segmente sind:

                    1. Kleinwagen – kleine Stadtautos, z.B. VW Up!, Renault Clio, Fiat Panda
                    2. Mittelklasse – normale Alltagsautos, z.B. VW Golf, Audi A4, BMW 3er
                    3. Geländewagen – große Fahrzeuge mit viel Platz, z.B. BMW X3, VW Tiguan, Ford Kuga, VW Multivan
                    4. Sportwagen – sportliche Fahrzeuge mit viel PS, z.B. Porsche 911, Audi R8, BMW M4
                    5. Luxusklasse – hochwertige Fahrzeuge mit Premiumausstattung, z.B. BMW 7er, Mercedes S-Klasse, Tesla Model S

                    Beispiele:
                    
                    Input: smart forTwo
                    Output: Kleinwagen

                    Input: Volkswagen Golf  
                    Output: Mittelklasse

                    Input: BMW X5  
                    Output: Geländewagen

                    Input: Porsche 911  
                    Output: Sportwagen

                    Input: Bentley Mulsanne
                    Output: Luxusklasse

                Gib mir als Output nur das Fahrzeugsegment aus.                             
                ''',
            ),
            ("human", "{input}"),
        ]
    )

    chain = prompt | llm
    res = chain.invoke(
        {
            "input": text
        }
    )

    return res.content


def use_llm_on_model(model_list):
    # Diese Funktion ruft die call_llm Funktion auf, um der Spalte model das Fahrzeugsegment zuzuordnen

    result = {}
    counter = 0
    limit_per_minute = 15 # brauchen diesen Timer um nicht zu viele Anfragen zu stellen (free sind 15 Anfragen/Min)

    for idx, model in enumerate(model_list, start=1):
        result[model] = call_llm(model)
        counter += 1

        # Warten, wenn das Limit erreicht ist
        if counter % limit_per_minute == 0:
            print(f"{counter} Anfragen gestellt – warte 60 Sekunden, um Rate Limit einzuhalten...")
            time.sleep(60)

    print("Fertig!")
    return result

In [3]:
df = pd.read_csv('data.csv')
df = df.iloc[:, 1:] 

In [10]:
X_train, X_test, y_train, y_test , X,y, categorical_features , numeric_features = preprocessing_pipeline()

FileNotFoundError: [Errno 2] No such file or directory: 'offer_description_utf8.csv'

In [ ]:
X

In [4]:
df.head()

,brand,model,color,registration_date,year,price_in_euro,power_kw,power_ps,transmission_type,fuel_type,fuel_consumption_l_100km,fuel_consumption_g_km,mileage_in_km,offer_description
0,alfa-romeo,Alfa Romeo GTV,red,10/1995,1995,1300,148,201,Manual,Petrol,"10,9 l/100 km",260 g/km,160500.0,2.0 V6 TB
1,alfa-romeo,Alfa Romeo 164,black,02/1995,1995,24900,191,260,Manual,Petrol,NaN,- (g/km),190000.0,"Q4 Allrad, 3.2L GTA"
2,alfa-romeo,Alfa Romeo Spider,black,02/1995,1995,5900,110,150,Unknown,Petrol,NaN,- (g/km),129000.0,ALFA ROME 916
3,alfa-romeo,Alfa Romeo Spider,black,07/1995,1995,4900,110,150,Manual,Petrol,"9,5 l/100 km",225 g/km,189500.0,2.0 16V Twin Spark L
4,alfa-romeo,Alfa Romeo 164,red,11/1996,1996,17950,132,179,Manual,Petrol,"7,2 l/100 km",- (g/km),96127.0,"3.0i Super V6, absoluter Topzustand !"


In [ ]:
model_list = df['model'].unique().tolist()
result = use_llm_on_model(model_list)
df["segment"] = df["model"].map(result).fillna("unknown")

In [ ]:
df.to_csv('df_mit_segment.csv', index=False)